# Data clean project 1: Xbox games

The aim of this project is to scrape the Xbox's games catalogue (https://www.xbox.com/en-AU/games/all-games), and create a clean dataset of games.

## Scraping

I used `beautifulSoup` and `selenium` to scrape the dymaic Xbox's games catalogue page.

In [70]:
import bs4, requests
from selenium import webdriver
import time
import pandas as pd, numpy as np
import os
import random

#%%SET UP SELENIUM DRIVER
driver = webdriver.Chrome(r'C:\Users\boydn\Desktop\Work and Study\Projects\chromedriver.exe')
driver.get('https://www.xbox.com/en-AU/games/all-games')
time.sleep(10)

One quirk I haven't found a reason for was that the `data-bi-name` attribute that allowed me to click through the pages was not present after running the driver. This caused an error in the code below and broke the entire script. I worked out a solution which was to scroll the pagination block and click a random thing in it. That seemed to do the trick.

In [ ]:
driver.execute_script("document.querySelector('ul.m-pagination').scrollIntoView()")
driver.find_element_by_css_selector('button#unique-id-for-paglist-generated-select-menu-trigger').click()

In [71]:
#%%SCRAPE

page = 1
links = []
product_id = []
release_date = []
ms_product = []
multiplayer = []
rating = []
price = []
name = []


while page <= 52:   
    soup = bs4.BeautifulSoup(driver.page_source, features='lxml')
    
    items = soup.select('div.m-product-placement-item')
    
    for item in items:
        product_id.append(item.get('data-bigid'))
        release_date.append(item.get('data-releasedate'))
        ms_product.append(item.get('data-msproduct'))
        multiplayer.append(item.get('data-multiplayer'))
        rating.append(item.get('data-rating'))
        price.append(item.get('data-listprice'))
    
    game_links = soup.select('a.gameDivLink')
    
    for game in game_links:
        links.append(game.get('href'))
        for child in game.children:
            if child.name == 'div':
                name.append(child.getText())
    page += 1
    page_css = 'a[data-bi-name="' + str(page) + '"]'
    
    try:
        elem = driver.find_element_by_css_selector(page_css)
    except:
        print('End page reached.')
        break

    elem.click()
    time.sleep(random.randint(1, 3))
    
game_data = {
    'product_id': product_id,
    'release_date': release_date,
    'name': name,
    'ms_product': ms_product,
    'rating': rating,
    'multiplayer': multiplayer,
    'link': links,
    'price': price
    }

os.chdir(r'C:\Users\boydn\Desktop\Work and Study\Projects\1. Data cleaning')
xbox = pd.DataFrame(game_data)

End page reached.


## Data wrangling

In [72]:
import pandas as pd, numpy as np

In [73]:
xbox.to_csv('xbox_data.csv')

In [75]:
xbox.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2572 entries, 0 to 2571
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   product_id    2572 non-null   object
 1   release_date  2572 non-null   object
 2   name          2572 non-null   object
 3   ms_product    2572 non-null   object
 4   rating        2572 non-null   object
 5   multiplayer   2572 non-null   object
 6   link          2572 non-null   object
 7   price         2572 non-null   object
dtypes: object(8)
memory usage: 160.9+ KB


In [78]:
xbox.head()
#xbox = xbox.drop('Unnamed: 0', axis=1)   #DROP COLUMN

,product_id,release_date,name,ms_product,rating,multiplayer,link,price
0,C54H8FDKTJPG,2017-12-19T00:00:00.0000000Z,Grand Theft Auto V: Premium Edition & Great Wh...,false,Restricted to 18+,true,https://www.xbox.com/en-au/games/gta-v,69.95
1,9N6FTC4QJMWN,2020-09-09T00:00:00.0000000Z,CarX Drift Racing Online$44.95,false,General,true,https://www.microsoft.com/en-au/p/carx-drift-r...,44.95
2,9NH019ZFFHTV,2021-08-12T00:00:00.0000000Z,Car Mechanic Simulator 2021$44.95,false,General,false,https://www.microsoft.com/en-au/p/car-mechanic...,44.95
3,9NV34NLCLG81,2017-10-02T13:00:00.0000000Z,Forza Motorsport 7 Ultimate EditionFull price ...,true,General,true,https://www.xbox.com/en-au/games/forza-7,29.98
4,9N2DMQBN9RC4,2020-12-16T18:00:00.0000000Z,Call of Duty®: Black Ops Cold War$99.95,false,Restricted,true,https://www.microsoft.com/en-au/p/call-of-duty...,99.95


Convert price to float type:

In [79]:
xbox['price'] = xbox['price'].astype(float) #CONVERT PRICE TO FLOAT TYPE

Process datetime values:

In [80]:
xbox['release_date'].isnull().value_counts() #NO MISSING RELEASE DATE

False    2572
Name: release_date, dtype: int64

In [81]:
xbox['release_date'] = pd.to_datetime(xbox['release_date'], errors='coerce')  #A FEW DATES WERE INVALID, SO USE COERCE

In [82]:
xbox['release_date'].isnull().sum()  #THE INVALID DATES ARE CONVERTED TO NaN, THERE ARE ONLY 4 OF THEM

4

The next step is to clean the game names: getting ride of the price and copyright symbols:

In [84]:
xbox.name[:15]

0     Grand Theft Auto V: Premium Edition & Great Wh...
1                        CarX Drift Racing Online$44.95
2                     Car Mechanic Simulator 2021$44.95
3     Forza Motorsport 7 Ultimate EditionFull price ...
4               Call of Duty®: Black Ops Cold War$99.95
5            Tom Clancy’s Ghost Recon® Breakpoint$99.95
6                                      Far Cry® 5$99.95
7             Grand Theft Auto V: Premium Edition$47.95
8             Fallout 4: Game of the Year Edition$54.95
9                  Assassin's Creed IV Black Flag$29.95
10    ASSASSIN'S CREED® VALHALLA - DELUXE EDITION$12...
11                                      Minecraft$29.95
12                           Rust Console Edition$74.95
13                         Madden NFL 22 Xbox One$99.95
14                                  Assetto Corsa$39.95
Name: name, dtype: object

In [102]:
#ONLY GAMES WITH DISCOUNTS HAVE THE "FULL PRICE WAS" TEXT AT THE END OF THE GAME, SO THE FIRST STEP IS TO TAKE THEM OUT
xbox['full_price'] = (xbox["name"].str.extract(r'(Full price.*?\$\d+\.\d+)', expand=False)
                                  .str.replace(r'Full price was $','', regex=False)
                                  .astype(float))

In [86]:
#WHAT'S LEFT IS THE CURRENT PRICE, I EXTRACTED THEM TO ANOTHER COLUMN NAMED new_price
xbox['new_price'] = (xbox["name"].str.extract(r'(\$\d+\.+\d+$)', expand=False)
                                 .str.replace('$', '', regex=False)
                                 .astype(float))

In [98]:
#REMOVE THE PRICE, "FULL PRICE" TEXT, AND ANY NON-ASCII SYMPBOLS FROM GAME NAME
xbox['name'] = (xbox["name"].str.replace(r'Full price.*', '', regex=True)
                            .str.replace(r'\$\d+\.\d+', '', regex=True)
                            .str.replace(r'[^\x00-\x7F]', '', regex=True))

In [99]:
#CALCULATE DISCOUNT PERCENT FOR GAMES ON SALE
xbox['discount_percent'] = np.round((xbox['new_price'] / xbox['full_price']) - 1, 2)

In [100]:
xbox['name']

0       Grand Theft Auto V: Premium Edition & Great Wh...
1                                CarX Drift Racing Online
2                             Car Mechanic Simulator 2021
3                     Forza Motorsport 7 Ultimate Edition
4                        Call of Duty: Black Ops Cold War
                              ...                        
2567                                         Unto The End
2568                                     Autumn's Journey
2569                              Do Not Feed the Monkeys
2570                                                I, AI
2571                         Lose Your Head Deluxe Bundle
Name: name, Length: 2572, dtype: object

## Final product

In [103]:
xbox

,product_id,release_date,name,ms_product,rating,multiplayer,link,price,full_price,new_price,discount_percent
0,C54H8FDKTJPG,2017-12-19 00:00:00+00:00,Grand Theft Auto V: Premium Edition & Great Wh...,false,Restricted to 18+,true,https://www.xbox.com/en-au/games/gta-v,69.95,NaN,69.95,NaN
1,9N6FTC4QJMWN,2020-09-09 00:00:00+00:00,CarX Drift Racing Online,false,General,true,https://www.microsoft.com/en-au/p/carx-drift-r...,44.95,NaN,44.95,NaN
2,9NH019ZFFHTV,2021-08-12 00:00:00+00:00,Car Mechanic Simulator 2021,false,General,false,https://www.microsoft.com/en-au/p/car-mechanic...,44.95,NaN,44.95,NaN
3,9NV34NLCLG81,2017-10-02 13:00:00+00:00,Forza Motorsport 7 Ultimate Edition,true,General,true,https://www.xbox.com/en-au/games/forza-7,29.98,NaN,29.98,-0.75
4,9N2DMQBN9RC4,2020-12-16 18:00:00+00:00,Call of Duty: Black Ops Cold War,false,Restricted,true,https://www.microsoft.com/en-au/p/call-of-duty...,99.95,NaN,99.95,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2567,9MVJ0W5N2L2Q,2020-12-09 00:00:00+00:00,Unto The End,false,Mature,false,https://www.microsoft.com/en-au/p/unto-the-end...,37.45,NaN,37.45,NaN
2568,9N2Z0W08GB5F,2020-12-09 00:00:00+00:00,Autumn's Journey,false,Mature,false,https://www.microsoft.com/en-au/p/autumns-jour...,7.45,NaN,7.45,NaN
2569,9N8QGLQSKR77,2020-12-09 00:00:00+00:00,Do Not Feed the Monkeys,false,Restricted,false,https://www.microsoft.com/en-au/p/do-not-feed-...,18.95,NaN,18.95,NaN
2570,9NT15BZXFHXF,2020-12-09 00:00:00+00:00,"I, AI",false,Parental guidance,false,https://www.microsoft.com/en-au/p/i-ai/9NT15BZ...,14.95,NaN,14.95,NaN
